In [14]:
import torch
from torch import nn
from tqdm import tqdm
from torchvision.datasets.mnist import FashionMNIST
from torchvision import transforms
import matplotlib.pyplot as plt

print("Using Device: ", torch.cuda.get_device_name())

Using Device:  Tesla T4


In [15]:
def create_datasets(data_root):
    trainset = FashionMNIST(root = data_root, train = True, download = True, transform = transforms.ToTensor())
    testset = FashionMNIST(root = data_root, train = False, download = True, transform = transforms.ToTensor())
    return trainset, testset

trainset, testset = create_datasets("./data")
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, num_workers=4)

In [37]:
class ConvBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding=0):
        super().__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
            nn.Tanh()
        )

    def forward(self, x):
        return self.conv_block(x)


class PoolingBlock(torch.nn.Module):
    def __init__(self, kernel_size, stride):
        super().__init__()
        self.pooling_block = nn.AvgPool2d(kernel_size=kernel_size, stride=stride)

    def forward(self, x):
        return self.pooling_block(x)

class LeNet5(torch.nn.Module):
    def __init__(self, n_channels=1):
        super().__init__()
        self.n_channels = n_channels

        self.conv1 = ConvBlock(in_channels=n_channels, out_channels=6, kernel_size=5, stride=1, padding=2)
        self.pooling1 = PoolingBlock(kernel_size=2, stride=2)
        self.conv2 = ConvBlock(in_channels=6, out_channels=16, kernel_size=5, stride=1)
        self.pooling2 = PoolingBlock(kernel_size=2, stride=2)
        self.conv3 = ConvBlock(in_channels=16, out_channels=120, kernel_size=5, stride=1)
        self.activation = nn.Tanh()
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.fc2 = nn.Linear(in_features=84, out_features=10)
        # self.dropout = nn.Dropout(p=0.5, inplace=True)
        self.softmax = nn.Softmax()
        # self.batch_norm = nn.BatchNorm1d()
        # self.batch_norm = nn.BatchNorm2d(num_features=)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pooling1(x)
        x = self.conv2(x)
        x = self.pooling2(x)
        x = self.conv3(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        # x = self.dropout(x)
        return self.softmax(x)  # numerically it is better to output raw logits and use the BCEwithLogitsLoss

class LeNet(nn.Module):
    def init(self):
        super(LeNet, self).init()
        self.cnn_model = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size = 5), #(N, 1, 28, 28) -> (N, 6, 24, 24)
            nn.Tanh(),
            nn.AvgPool2d(2, stride = 2), #(N, 6, 24, 24) -> (N, 6, 12, 12)
            nn.Conv2d(6, 16, kernel_size = 5), #(N, 6, 12, 12) -> (N, 6, 8, 8)
            nn.Tanh(),
            nn.AvgPool2d(2, stride = 2)) #(N, 6, 8, 8) -> (N, 16, 4, 4)
        self.fc_model = nn.Sequential(
            nn.Linear(256, 120), # (N, 256) -> (N, 120)
            nn.Tanh(),
            nn.Linear(120, 84), # (N, 120) -> (N, 84)
            nn.Tanh(),
            nn.Linear(84, 10))  # (N, 84)  -> (N, 10)) #10 classes
    def forward(self, x):     
        x = self.cnn_model(x)     
        x = x.view(x.size(0), -1)     
        x = self.fc_model(x)     
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# net = LeNet5().to(device)
net = LeNet().to(device)

In [38]:
def train_epoch(net, dataloader, loss_func, optimizer):
    net.train()
    total_batches = len(dataloader)
    all_losses = []
    with tqdm(total=total_batches) as pbar:
        for batch in dataloader:
            optimizer.zero_grad()

            images, labels = batch
            images, labels = images.to(device, dtype=torch.float), labels.to(device, dtype=torch.long)
            
            est_labels = net(images)
            loss = loss_func(est_labels, labels)
            loss.backward()
            optimizer.step()
            all_losses.append(loss.item())

            pbar.update(1)
    return all_losses

def compute_accuracy(est_labels, labels):
    correct_pred = torch.sum(est_labels == labels)
    total = labels.shape[0]
    return correct_pred / total

def eval(net, dataloader, loss_func):
    net.eval()
    total_batches = len(dataloader)
    all_losses = []
    all_acc = []
    with torch.no_grad():
        with tqdm(total=total_batches) as pbar:
            for batch in dataloader:
                images, labels = batch
                images, labels = images.to(device, dtype=torch.float), labels.to(device, dtype=torch.long)
            
                est_labels = net(images)
                all_losses.append(loss_func(est_labels, labels).item())
                all_acc.append(compute_accuracy(torch.argmax(est_labels, dim=1), labels))

                pbar.update(1)

    return all_losses, all_acc

def train(net, train_loader, val_loader, num_epochs, loss_func, optimizer):
    all_train_losses = []
    all_val_losses = []
    all_val_acc = []
    for epoch in range(num_epochs):
        print("Training Epoch %d / %d" % (epoch, num_epochs))
        train_losses = train_epoch(net, train_loader, loss_func, optimizer)
        print("Epoch %d train loss: %f" % (epoch, sum(train_losses) / len(train_losses)))
        all_train_losses.append(train_losses)
        val_losses, val_acc = eval(net, val_loader, loss_func)
        print("Epoch %d val loss: %f" % (epoch, sum(val_losses) / len(val_losses)))
        print("Epoch %d val acc: %f" % (epoch, sum(val_acc) / len(val_acc)))
        all_val_losses.append(val_losses)
        all_val_acc.append(val_acc)
        
    return all_train_losses, all_val_losses, all_val_acc
        


In [39]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
loss_func = torch.nn.CrossEntropyLoss()
num_epochs = 10
all_train_losses, all_val_losses, all_val_acc = train(net, train_loader, test_loader, num_epochs, loss_func, optimizer)

ValueError: ignored